In [2]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.models import model_from_json
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard
from time import time
import os
import pypianoroll as pr
import numpy as np
import random

/Users/hsureggie/coding/python/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import tensorflow as tf

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

tf.keras.backend.set_session(sess)


In [4]:
def load_model():
    # load JSON and create model
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model.h5")
    print("Loaded model from disk")
    loaded_model.compile(loss=keras.losses.binary_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return loaded_model
loaded_model = load_model()

Loaded model from disk


In [17]:
#handle the prediction returned from the model
def handle_prediction(prediction,song,random,length):
    label = []
    if random != 0:
        prediction = prediction.reshape(int(prediction.shape[0]/random),random,1)
    elif random == 0:
        prediction = prediction.reshape(int(prediction.shape[0]/length),length,1)
    avg_score = np.average(prediction,axis=1)>0.5
    for x in range(len(song.tracks)):
        if avg_score[x]:
            song.tracks[x].name = "Melody"
    return song

In [24]:
def predict_song(root,name,loaded_model,random):
    img_rows = 128
    img_cols = 500
    store = []
    temp = []
    full_name = os.path.join(root,name)
    if full_name.endswith('.npz'):
        temp_song = pr.load(full_name)
        #iterate over tracks and randomly sample 3 subarrays in a track if random is set
        for track in temp_song.tracks:
            if random != 0:
                random_ints = random.sample(range(0,track.pianoroll.shape[0]-500),random)
                for x in range(0,random):
                    temp_array = np.array(track.pianoroll[random_ints[x]:random_ints[x]+500,:])
                    temp.append((temp_array).swapaxes(0,1))
            elif random == 0:
                for x in range(0,int(track.pianoroll.shape[0]/500)):
                    temp_array = np.array(track.pianoroll[500*x:500*(x+1),:])
                    temp.append((temp_array).swapaxes(0,1))
                length = len(temp)
            temp = np.array(temp)
            temp = temp.reshape(temp.shape[0],img_rows,img_cols,1)
            store.extend(temp)
            temp = []
        store = np.array(store)
        prediction = loaded_model.predict(store,verbose=1)
        after_song = handle_prediction(prediction,temp_song,random,length=length)
    return prediction,after_song

In [23]:
files_dealt = 0
for root,dirs,files in os.walk("./lpd_cleansed/", topdown=False):
    for name in files:
        full_name = os.path.join(root,name)
        print(full_name)
        try:
            prediction, labeled_song = predict_song(root,name,loaded_model,random = 0)
            path = "./labeled_1/"+"/".join(full_name.split('/')[2:])
            print(path)
            directory = "/".join(path.split('/')[:-1])
            if not os.path.exists(directory):
                os.makedirs(directory)
            labeled_song.save(path)
            print(str(files_dealt+1)+" songs cleansed")
        except Exception as error:
            print(files_dealt+1)
            print("Stucked file: "+ full_name)
            raise(error)
            pass
        files_dealt+=1


./lpd_cleansed/R/R/U/TRRRUTV12903CEA11B/a8a167f3ab4d81560b65095e7565a837.npz
(17, 128, 500, 1)
(17, 128, 500, 1)
(17, 128, 500, 1)
(17, 128, 500, 1)
(17, 128, 500, 1)
(17, 128, 500, 1)
(17, 128, 500, 1)
(17, 128, 500, 1)
(17, 128, 500, 1)
153/153 [==============================] - 17s 110ms/step
./labeled/R/R/U/TRRRUTV12903CEA11B/a8a167f3ab4d81560b65095e7565a837.npz
1 songs cleansed
